## Build earnings reports (OO version)

E. Quinn 12/27/2020

This notebook uses the local SC classes to build objects from the MUNIS earnings reports

## Import standard python datascience packages

In [1]:
import sys
import math
import re
import copy
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [2]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

In [3]:
sys.path.append("/home/gquinn/EG/school_committee/egsc/py_egsc")

### Import custom SC classes

In [4]:
from person import Person
from ucoa_labels import UCOA_labels
from eg_acct_codes import EG_acct_codes
from pay_check import Pay_check, Check_lineitem
from roles import *
from scenario import Scenario
from salaries import *
from payperiod import Payperiod

### Show the directory we are running in

In [5]:
!pwd

/home/gquinn/EG/school_committee/finance_subcommittee/notebooks


### Load RIDE UCOA labels 

In [6]:
ucoa_labels = UCOA_labels()
help(ucoa_labels)

Help on UCOA_labels in module ucoa_labels object:

class UCOA_labels(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      UCOA_labels class provides labels for UCOA fields
 |  
 |  get_label(self, col, code)
 |      Usage: get_label(col,code) returns the code for field col
 |  
 |  get_labels_dictionary(self)
 |      Returns the UCOA labels dictionary
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



### EG accounting codes class

provides descriptions for EG accounting codes and mapping to UCOA codes

In [7]:
eg_acct_codes = EG_acct_codes()
help(eg_acct_codes)

Help on EG_acct_codes in module eg_acct_codes object:

class EG_acct_codes(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  check_eg_acct_desc6(self, acct)
 |      check whether first 6 digits map an acct6 key
 |  
 |  get_UCOA_from_acct(self, acct, ucl)
 |  
 |  get_UCOA_from_acct6(self, acct6, ucl)
 |  
 |  get_eg_acct_UCOA(self, acct)
 |      Provides UCOA codes for accounting codes in EG MUNIS system.
 |  
 |  get_eg_acct_codes(self)
 |      Returns dictionary of account codes.
 |  
 |  get_eg_acct_desc(self, acct)
 |      Provides descriptions for accounting codes in EG MUNIS system.
 |  
 |  get_eg_acct_desc6(self, acct)
 |      Provides descriptions for accounting codes in EG MUNIS system.
 |  
 |  get_loc_from_acct(self, acct, ucl)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary 

### Teacher salary matrix

In [8]:
salaries = Teacher_salary()
help(salaries)

pararates = Para_rates()
help(pararates)

facilities_rates = Facilities_rates()
help(facilities_rates)

Help on Teacher_salary in module salaries object:

class Teacher_salary(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_cba_matrix(self)
 |  
 |  get_colname(self, col_index)
 |  
 |  get_salary(self, school_year, col, step)
 |      Returns CBA salary given school year, column.
 |  
 |  get_school_year_cba_matrix(self, school_year)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

Help on Para_rates in module salaries object:

class Para_rates(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_cba_matrix(self)
 |  
 |  get_school_year_cba_matrix(self, school_year)
 |  
 |

### Read ll structure from pickle

In [9]:
#Load data (deserialize)
with open('../../finance_subcommittee/ll_12_23_2020.pkl', 'rb') as handle:
    ll = pickle.load(handle)
    
len(ll)

6

### Build a dictionary of people

In [10]:
people = {}

for year in ll.keys():
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            names         = ll[year][page][tb]['names']
            positions     = ll[year][page][tb]['positions']
            for i in np.arange(len(names)):
                name            = names[i]
                if (name not in people.keys()):
                    people[name] = Person(name)

len(people)

898

### Add roles, payperiods, checks, and lineitems

In [11]:
for year in ll.keys():
    for page in ll[year].keys():
        for tb in ll[year][page].keys():
            check_numbers = ll[year][page][tb]['checks']
            names         = ll[year][page][tb]['names']
            check_dates   = ll[year][page][tb]['dates']
            fund          = ll[year][page][tb]['fund']
            acct          = ll[year][page][tb]['acct']
            obj           = ll[year][page][tb]['obj']
            positions     = ll[year][page][tb]['positions']
            rates         = ll[year][page][tb]['rates']
            earnings      = ll[year][page][tb]['earnings']
            obj_desc      = ucoa_labels.get_label('Obj',obj)
            acct_desc     = eg_acct_codes.get_eg_acct_desc(acct)
            acct_UCOA     = eg_acct_codes.get_eg_acct_UCOA(acct)
            
            for i in np.arange(len(names)):
                check_number    = check_numbers[i]
                name            = names[i]
                date_str        = check_dates[i]
                position        = positions[i]
                rate            = rates[i]
                earned          = earnings[i]
                roles           = people[name].get_roles()
                if position not in roles.keys():
                    if (position == 'TEACHER'):
                        role = Teacher(name)
                    elif (position == 'PARAPROFES'):
                        role = Para(name)
                    elif (position == 'SPED PARAS'):
                        role = Sped_Para(name)
                    elif ('T/S' in position):
                        role = Substitute_teacher(name)
                    elif ('P/S' in position):
                        role = Substitute_para(name)
                    elif (position in ['YR SECRETA','SECRETARY']):
                        role = Office(name)
                    elif (position in ['CUSTODIAN','MAINTENANC','ELECTRICAN', \
                        'CUST PT','DIR MAINT','FACILITY DR']):
                        role = Facilities(name)
                    elif (position == 'COACH'):
                        role = Coach(name)
                    else:
                        role = Role(name)

                    people[name].add_role(position,role)
                role = people[name].get_role(position)
                date_str        = check_dates[i]
                words = date_str.split('/')
                check_date   = date(int(words[2]),int(words[0]),int(words[1]))
                
                pperiod = Payperiod(check_date)
                schyr = pperiod.get_school_year()
                syseq = pperiod.get_school_year_seq()
                if not role.check_payperiod(schyr,syseq):
                    role.add_payperiod_by_index(schyr,syseq,pperiod)
                else:
                    pperiod = role.get_payperiod(schyr,syseq)
                chk = pperiod.get_check(check_number)
                if chk is None:
                    chk = Pay_check(check_number,name,check_date)
                    pperiod.add_check(check_number,chk)
                litem =  Check_lineitem(fund,acct,obj,position,rate, \
                        earned,acct_desc,obj_desc)
                
                if (position == 'TEACHER'):
                    salary_matrix = salaries.get_school_year_cba_matrix(schyr)
                    role.decode_earnings(pperiod,chk,litem,salary_matrix)
                    
                elif (position in ['PARAPROFES','SPED PARAS']):
                    salary_matrix = pararates.get_school_year_cba_matrix(schyr)
                    role.decode_earnings(pperiod,chk,litem,salary_matrix)
                    
                elif (position in ['SECRETARY','YR SECRETA']):
                    salary_matrix = pararates.get_school_year_cba_matrix(schyr)
                    role.decode_earnings(pperiod,chk,litem,salary_matrix)
                     
                elif (position in ['CUSTODIAN','MAINTENANC','ELECTRICAN']):
                    
                    salary_matrix = facilities_rates.get_cba_matrix_by_year(schyr)
                    role.decode_earnings(pperiod,chk,litem,salary_matrix)
                    
                chk.add_item(litem)
                    

len(people)

898

In [12]:
with open('../../finance_subcommittee/people_12_27_2020.pkl', 'wb') as handle:
    pickle.dump(people, handle)